# Trabajo Práctico 1: Enunciado


El trabajo práctico 1 de la materia se basa en el análisis de los tweets del set de datos de la competencia: https://www.kaggle.com/c/nlp-getting-started.  

- id - identificador unico para cada  tweet
- text - el texto del tweet
- location - ubicación desde donde fue enviado (podría no estar)
- keyword - un keyword para el tweet  (podría faltar)
- target - en train.csv, indica si se trata de un desastre real  (1) o no (0)

El objetivo del primer TP es realizar un análisis exploratorio del set de datos. Queremos ver qué cosas podemos descubrir sobre los datos que puedan resultar interesantes. Estas cosas pueden estar relacionadas al objetivo del TP2 (predecir si un cierto tweet es real o no) o no, ambas son de interés.

Los requisitos de la primera entrega son los siguientes:

- El análisis debe estar hecho en Python Pandas o R.
- El análisis debe entregarse en formato pdf vía gradescope. En el informe no va código.
- Informar el link a un repositorio Github en donde pueda bajarse el código completo para generar el análisis.

La evaluación del TP se realizará en base al siguiente criterio:

- Originalidad del análisis exploratorio. 
- Calidad del reporte. ¿Está bien escrito? ¿Es claro y preciso? 
- Calidad del análisis exploratorio: qué tipo de preguntas se hacen y de qué forma se responden, ¿es la respuesta clara y concisa con respecto a la pregunta formulada? 
- Calidad de las visualizaciones presentadas.
  - ¿Tienen todos los ejes su rótulo?
  - ¿Tiene cada visualización un título?
  - ¿Es entendible la visualización sin tener que leer la explicación?
  - ¿El tipo de plot elegido es adecuado para lo que se quiere visualizar?
  - ¿Es una visualización interesante?
  - ¿El uso del color es adecuado?
  - ¿Hay un exceso o falta de elementos visuales en la visualización elegida?
  - ¿La visualización es consistente con los datos?
- Conclusiones presentadas.


In [ ]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs


In [ ]:
tweets = pd.read_csv('../data/train.csv', encoding='utf-8')
tweets.info()

In [ ]:
# keyword tiene 61 nulls
# location tiene 2533 nulls

In [ ]:
tweets.location.value_counts() # se podría unificar varios a USA

### Análisis de los falsos

In [ ]:
tweets[tweets["target"] == 0]["text"] #los falsos

In [ ]:
# primera normalización del texto, pasamos a lowercase 
tweets['normalized_text'] = tweets.text.str.lower()

#### Eliminamos los links del texto y dejandolos en una columna a parte por si sirven a futuro

In [ ]:
# nueva columna con el total de links
import re
URLPATTERN = r'(https?://\S+)' 

tweets['urls'] = tweets.normalized_text.apply(lambda x: re.findall(URLPATTERN, x))
tweets['normalized_text'] = tweets.normalized_text.apply(lambda x: re.sub(URLPATTERN,"", x))
# cuento la cantidad de links en los tweets
tweets['url_count'] = tweets.urls.str.len()

In [ ]:
tweets[tweets["target"] == 1].keyword.value_counts() #los posta, keywords

In [ ]:
# vemos como queda el dataset
tweets[(tweets['url_count'] > 1)].head()

#### Eliminando palabras que no son super importantes

In [ ]:
!pip3 install nltk
!pip3 install stopwords

In [ ]:
#import nltkfrom nltk.corpus 
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize



In [ ]:
stop_words = stopwords.words('english')
stop_words[1:15]

In [ ]:
# cuento la cantidad de hashtags en los tweets
# nueva columna con el total de hashtags, y los hashtags
tweets['hashtags'] = tweets.normalized_text.apply(lambda x: re.findall(r"#(\w+)", x))
tweets['hashtags_count'] = tweets.hashtags.str.len()
tweets['normalized_text'] = tweets.normalized_text.apply(lambda x: re.sub(r"#(\w+)","", x))

In [ ]:
# vemos como queda el dataset
tweets[(tweets['hashtags_count'] > 1)].head()

#### Eliminamos los tags del texto y lo dejamos en otra columna junto con el count

In [ ]:
# cuento la cantidad de ags en los tweets
# nueva columna con el total de tags, y los tags
tweets['tags'] = tweets.text.str.lower().apply(lambda x: re.findall(r"@(\w+)", x))
tweets['normalized_text'] = tweets.normalized_text.apply(lambda x: re.sub(r"@(\w+)","", x))
tweets['tags_count'] = tweets.tags.str.len()

In [ ]:
# vemos como queda el dataset
tweets[(tweets['tags_count'] > 1)].head()

#### Contador de links (DESCARTADO)
Quiero ver si hay relación entre cantidad de tweets fakes y no fakes y la cantidad de links (DESCARTADO)

In [ ]:
# cuento la cantidad de links en los tweets
import re
URLPATTERN = r'(https?://\S+)' 
# nueva columna con el total de links
tweets['url_count'] = tweets.text.apply(lambda x: re.findall(URLPATTERN, x)).str.len()
tweets.head()

In [ ]:
tweets[tweets["target"] == 1].url_count.value_counts() #los posta, cantidad de urls

In [ ]:
tweets[tweets["target"] == 0].url_count.value_counts() #los fake, cantidad de urls

In [ ]:
# grafico esto para ver si hay relación
g = sns.boxplot(x="target", y="url_count", 
                     data=tweets, palette="cubehelix")
g.set_title("Cantidad de links según veracidad", fontsize=18)
g.set_xlabel("Real?", fontsize=14)
g.set_ylabel("Cantidad de links", fontsize=14)

In [ ]:
import string
translator = str.maketrans('', '', string.punctuation)

# !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~

def remove_punctuation(text):        
    return text.translate(translator)

In [ ]:
!pip install bs4

In [ ]:
from bs4 import BeautifulSoup
from html import unescape

In [ ]:
def remove_htmlsymbols(text):
    soup = BeautifulSoup(unescape(text))
    return soup.text

In [ ]:
#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

def remove_emojis_non_ascii(text):    
    #replace consecutive non-ASCII characters with a space
    result = re.sub(r'[^\x00-\x7F]+',' ', text)
    #remove emojis from tweet
    result = emoji_pattern.sub(r'', result)    
    return result

In [ ]:
tweets['normalized_text'] = tweets.normalized_text.apply(remove_htmlsymbols)
tweets['normalized_text'] = tweets.normalized_text.apply(remove_punctuation)
tweets['normalized_text'] = tweets.normalized_text.apply(remove_emojis_non_ascii)

In [ ]:
#### Eliminamos stop words y creamos nueva columna con array de palabras

In [ ]:
!pip install nltk
!pip install stopwords

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

In [ ]:
# convertimos el texto en listado de palabras y despues borramos las stop words
tweets['words'] = tweets.normalized_text.str.split()
stop_words = stopwords.words('english')
tweets['normalized_words'] = tweets['words'].apply(lambda x: [item for item in x if item not in stop_words])
# eliminar las stop words del texto normalizado
tweets['normalized_text'] = [' '.join(map(str, l)) for l in tweets['normalized_words']]
# vemos como queda el dataset
tweets.head()

In [ ]:
# todos los tags
tweets.text.str.extractall(r'(\@\w+)')[0].value_counts()


In [ ]:
tweets[tweets["target"] == 1].tags_count.value_counts() #los posta

In [ ]:
tweets[tweets["target"] == 0].tags_count.value_counts() #los fake}

In [ ]:
# grafico esto para ver si hay relación
g = sns.boxplot(x="target", y="tags_count", 
                     data=tweets, palette="cubehelix")
g.set_title("Cantidad de tags según veracidad", fontsize=18)
g.set_xlabel("Real?", fontsize=14)
g.set_ylabel("Cantidad de tags", fontsize=14)

#### Analizamos si el texto tiene números y guardamos el dato en una nueva columna

In [ ]:
tweets.corr()

### Análisis de location

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Analisis sobre cuestiones vistas despues de la reunion con Vir.

- Analisis de Tweets c/location vs Tweets s/location
- Keywords / Palabras y veracidad
- Relacion entre location - existencia en el texto


### Analisis de Tweets c/location vs tweets s/location

In [ ]:
#cantidad de tweets con y sin location
tweets['location'].isna().value_counts()

In [ ]:
#tweets reales y no
tweets['target'].value_counts()

In [ ]:
#relacion entre ambos datos
tweets['has_location'] = ~tweets['location'].isna()
tweets[['has_location','target']].groupby(['target']).count()
#no aporta nada.

In [ ]:
tweets['keyword'] = tweets['keyword'].str.replace('\s','%20')

In [ ]:
#"apariciones / total de tweets" va a dar una aproximacion mejor de la ponderacion de las palabras sobre los tweets
total_keyword = tweets['keywords'].value_counts().count()
total_keyword

In [ ]:
tweets[tweets["target"] == 0]["keyword"].value_counts()

In [ ]:
keyword_false = tweets[tweets["target"] == 0]["keyword"].value_counts()
keyword_true  = tweets[tweets["target"] == 1]["keyword"].value_counts()
#keyword_false.index = keyword_false.index.str.replace('\s','%20')
keyword_false

In [ ]:
#relacion entre keywords y verac

g = sns.barplot(x=keyword_false.head(20).values,y=keyword_false.head(20).index.str.replace('%20',' '),color='red',orient="h").set(title = 'Top 30 keywords (Tweets falsos)',xlabel='Apariciones')


In [ ]:
#keyword_false.merge(keyword_real)
f = sns.barplot(x=keyword_true.head(20).values,y=keyword_true.head(20).index,color='green',orient="h").set(title = 'Top 30 keywords (Tweets verdaderos)',xlabel='Apariciones')

In [ ]:
#comparacion entre datasets?. buscar palabras en tops. - pendiente comparacion entre dos datasets
comparewords = pd.concat([keyword_true,keyword_false], axis=1)
comparewords.columns=['valid','invalid']

comparewords = comparewords.fillna(0)
comparewords = comparewords.sort_values(by=['valid'])

#comparewords_ds.plot.barh(rot=0)
comparewords.tail(15).plot.barh(rot=0).set(title='descartar?')


In [ ]:
comparewords = comparewords.sort_values(by=['invalid'])
comparewords.tail(15).plot.barh(rot=0).set(title='descartar?')


### Relacion entre location y existencia en el texto.
por lo visto es un dato irrelevante que se podria descartar. ya que la location puede ser cualquier cosa y no tiene nada que ver con el lugar de donde se tuitea


In [ ]:
tweets = tweets.fillna('None')

In [ ]:
#casos excluidos (si no tienen location, obviamente no van a estar en el texto)
tweets['text_in_location'] = tweets[['text','location']].apply(lambda row: 'Existe' if row.location in row.text else 'No existe' ,axis = 1)
tweets['text_in_location'].value_counts()


In [ ]:
count_location = tweets['text_in_location'].value_counts()

count_location.plot.bar(rot = 0).set(title = 'Correspondencia de location y su existencia en el texto',ylabel='cantidad tweets')
#g = sns.boxplot(x="target", y="url_count", 
#                     data=tweets, palette="cubehelix")
#g.set_title("Cantidad de links según veracidad", fontsize=18)
#g.set_xlabel("Real?", fontsize=14)
#g.set_ylabel("Cantidad de links", fontsize=14)

##### **No tiene mucho sentido hacer una clasificacion de tweets reales/falsos dentro de la correspondencia de location/texto , ya que es muy poca la relacion y no nos dice nada.**





In [ ]:
#Graficos sobre series numericas.
#tweets[tweets["target"] == 1].select_dtypes('number')

In [ ]:
#Pendiente
tweets['url_len'] = tweets.urls.astype(str).map(len)
tweets['text_len'] = tweets.text.map(len)

ax = sns.kdeplot(tweets[tweets["target"] == 0].text_len,\
                 tweets[tweets["target"] == 0].url_len *100/ \
                 tweets[tweets["target"] == 0].text_len , \
                 cmap="Reds", shade=True, shade_lowest=False, alpha=0.7)
ax = sns.kdeplot( tweets[tweets["target"] == 1].text_len,\
                 tweets[tweets["target"] == 1].url_len *100/ \
                 tweets[tweets["target"] == 1].text_len, \
                 cmap="Greens", shade=True, shade_lowest=False, alpha=0.7)
ax

In [ ]:
#Pendiente
tweets['hashtags_len'] = tweets.hashtags.astype(str).map(len)
tweets['tags_len'] = tweets.tags.astype(str).map(len)
tweets['words_len'] = tweets.words.astype(str).map(len)
tweets['normalized_words_len'] = tweets.normalized_words.astype(str).map(len)

ax = sns.kdeplot(tweets[tweets["target"] == 0].text_len,\
                 tweets[tweets["target"] == 0].hashtags_len *100/ \
                 tweets[tweets["target"] == 0].text_len , \
                 cmap="Reds", shade=True, shade_lowest=False, alpha=0.7)
ax = sns.kdeplot( tweets[tweets["target"] == 1].text_len,\
                 tweets[tweets["target"] == 1].hashtags_len *100/ \
                 tweets[tweets["target"] == 1].text_len, \
                 cmap="Greens", shade=True, shade_lowest=False, alpha=0.7)
ax


In [ ]:
ax = sns.kdeplot(tweets[tweets["target"] == 0].text_len,\
                 tweets[tweets["target"] == 0].tags_len *100/ \
                 tweets[tweets["target"] == 0].text_len , \
                 cmap="Reds", shade=True, shade_lowest=False, alpha=0.7)
ax = sns.kdeplot( tweets[tweets["target"] == 1].text_len,\
                 tweets[tweets["target"] == 1].tags_len *100/ \
                 tweets[tweets["target"] == 1].text_len, \
                 cmap="Greens", shade=True, shade_lowest=False, alpha=0.7)
ax


In [ ]:
ax = sns.kdeplot(tweets[tweets["target"] == 0].text_len,\
                 tweets[tweets["target"] == 0].words_len *100/ \
                 tweets[tweets["target"] == 0].text_len , \
                 cmap="Reds", shade=True, shade_lowest=False, alpha=0.7)
ax = sns.kdeplot( tweets[tweets["target"] == 1].text_len,\
                 tweets[tweets["target"] == 1].words_len*100/ \
                 tweets[tweets["target"] == 1].text_len, \
                 cmap="Greens", shade=True, shade_lowest=False, alpha=0.7)
ax


In [ ]:
ax = sns.kdeplot(tweets[tweets["target"] == 0].text_len,\
                 tweets[tweets["target"] == 0].normalized_words_len *100/ \
                 tweets[tweets["target"] == 0].text_len , \
                 cmap="Reds", shade=True, shade_lowest=False, alpha=0.7)
ax = sns.kdeplot( tweets[tweets["target"] == 1].text_len,\
                 tweets[tweets["target"] == 1].normalized_words_len*100/ \
                 tweets[tweets["target"] == 1].text_len, \
                 cmap="Greens", shade=True, shade_lowest=False, alpha=0.7)
ax


In [ ]:
#Pendiente
text_analytics = pd.DataFrame()

text_analytics['url_len'] = tweets.urls.astype(str).map(len)
text_analytics['text_len'] = tweets.text.map(len)
text_analytics['hashtags_len'] = tweets.hashtags.astype(str).map(len)
text_analytics['tags_len'] = tweets.tags.astype(str).map(len)
text_analytics['words_len'] = tweets.words.astype(str).map(len)
text_analytics['normalized_words_len'] = tweets.normalized_words.astype(str).map(len)
text_analytics['target'] = tweets[['target']].apply(lambda row: row.target==1 ,axis = 1)

gsnum = sns.pairplot(text_analytics,hue="target",diag_kind="kde").set(title='descartar')


In [ ]:

#g = sns.distplot(countUrl[['valid']], color="green",label="valid",hist=False,rug=False)

### analisis en base a la cantidad de caracters
- lenght vs hashtag-
- lenght vs url-
- lenght vs tags-

(ya lo vio Vir, en todo caso evaluar la visualizacion y si sirve el Distplot)


### Analisis de keywords - PARTE 2

algunas palabras similares tienen distintos indices de aparicion. Analizo el coeficiente (valido e invalido) para las palabras ordenadas alfabeticamente

In [ ]:
#Tabla pivot para ver las apariciones(unicas) en cada tipo de tweet
keyword_in_target = pd.pivot_table(
tweets[['id', 'keyword', 'target']], 
index='keyword',columns='target',aggfunc=lambda x: len(x.unique()),fill_value=0
)
keyword_in_target.reset_index()
#rename columnas
keyword_in_target.columns= ['valid','invalid']
#agrego columna total y calculo promedio
keyword_in_target['total'] = keyword_in_target[['valid','invalid']].apply(lambda row: row.valid+ row.invalid ,axis = 1)

def averageshow(args):
    valid =args[0]/args[2]
    invalid = args[1]/args[2] 
    return pd.Series([valid,invalid])

a = keyword_in_target[['valid','invalid','total']].apply(averageshow,axis=1)
a.reset_index()
a.columns=['valid','invalid']
a.head(10).plot.barh(rot=0)


In [ ]:
a.tail(10).plot.barh(rot=0)

### NOTA:
podria armar algo similar con la columna Words y ver si hay algo interesante.


### Analisis sobre la columna location - Parte 2

In [ ]:
location_bar_true = tweets[tweets["target"] == 0]['location'].value_counts().reset_index()
location_bar_false = tweets[tweets["target"] == 1]['location'].value_counts().reset_index()

#location_bar_df.columns = ['location', 'count']
location_bar = pd.concat([location_bar_true.set_index('index'), location_bar_false.set_index('index')], axis=1, join='outer')
location_bar.columns = ['valid','invalid']
location_bar.head(10).plot.barh(rot=0)
location_bar.tail(10).plot.barh(rot=0)

**antes debo limpiar la data en Location**


# Analisis simple.

Analizo la cantidad de palabras (token_len)  y lo analizo en relacion del largo del tweet (text_len
Creo que daria el mismo resultado contando las palabras de Words o Normalized_words

In [ ]:
#dropna=False

text_simple = pd.DataFrame()
text_simple['cantidad_palabras'] = tweets['text'].apply(lambda x: len([token for token in x.split()]))
text_simple['largo_texto'] = tweets['text'].apply(lambda x: len(x))
text_simple['estado'] = tweets[['target']].apply(lambda row: row.target==1 ,axis = 1)

g = sns.FacetGrid(text_simple, col="estado",height=5)
g = g.map(sns.distplot, "largo_texto").set(ylabel="densidad")
plt.show()



In [ ]:

g = sns.FacetGrid(text_simple, col="estado",height=5)
g = g.map(sns.distplot, "cantidad_palabras").set(ylabel="densidad")
plt.show()

In [ ]:
text_simple

In [ ]:
#Relacion entre cantidad de palabras y largo de texto
ax = sns.kdeplot(text_simple[text_simple["estado"]==False].cantidad_palabras,                  
                 text_simple[text_simple["estado"]==False].largo_texto, 
                 cmap="Reds", shade=True, shade_lowest=False, alpha=0.7).set(title='Largo texto vs Cantidad de palabras (Tweets falsos)')
ax

In [ ]:
az = sns.kdeplot(text_simple[text_simple["estado"]==True].cantidad_palabras,                  
                 text_simple[text_simple["estado"]==True].largo_texto, 
                 cmap="Greens", shade=True, shade_lowest=False, alpha=0.7).set(title='Largo texto vs Cantidad de palabras (Tweets reales)')
az

### Spellchecker
Es un corrector de escritura, se podria aplicar a los tweets para limpiar la data


In [ ]:
!pip3 install pyspellchecker

In [ ]:
#Muy lento

#from spellchecker import SpellChecker

#spell = SpellChecker()
#def correct_spellings(text):
#    corrected_text = []
#    misspelled_words = spell.unknown(text.split())
#    for word in text.split():
#        if word in misspelled_words:
#            corrected_text.append(spell.correction(word))
#        else:
#            corrected_text.append(word)
#    return " ".join(corrected_text)
 

#correct_spellings('')

#tweets['normalized_text']=tweets['normalized_text'].apply(lambda x : correct_spellings(x))

#tweets


### otras cosillas
Esto en general va arriba. la distribucion de tweets falsos y verdaderos

In [ ]:

# extracting the number of examples of each class
Real_len = tweets[tweets['target'] == 1].shape[0]
Not_len = tweets[tweets['target'] == 0].shape[0]

# bar plot of the 3 classes
plt.rcParams['figure.figsize'] = (7, 5)
plt.bar(10,Real_len,3, label="Real", color='green')
plt.bar(15,Not_len,3, label="Falso", color='red')
plt.legend()
plt.ylabel('Numero de muestras')
plt.title('Distribución de tweets')
plt.show()